In [2]:
# Packages
using Pkg
Pkg.activate(".")
Pkg.instantiate()
using CUDA
using Flux
using Statistics
using ProgressMeter
using Plots

  Activating project at `~/src_nobackup/julia_ml_tests.jl.git`


In [3]:
# regular computations with the Flux.gpu and Flux.cpu functions
a=randn(5,1000)
a_gpu = a |> gpu
b=ones(5,1000)
b_gpu = b |> gpu
c_gpu = sum(a_gpu.*b_gpu,dims=2)
c = c_gpu |> cpu
typeof(c),typeof(c_gpu)

(Matrix{Float32}, CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})

In [4]:
#Note that the x |> gpu is the same as Flux.gpu(x)
# the |> operator is called the pipe operator and is generic in Julia
x=2.0
f(x)=x^2
y = x |> f #but it looks cooler than f(x)
y

4.0

In [15]:
m = Dense(2=>3, relu)
#@which m(ones(2))
println("w = $(m.weight)")
println("b = $(m.bias)")
println("activation = $(m.σ)")
# apply
x = rand(Float32,2)
y = m(x)
# apply to batch
x = rand(Float32,2, 4)
y = m(x)
# get trainable parameters
p=Flux.params(m)
println("p = $(p)")

w = Float32[0.9782329 -0.32619792; -0.11098309 -0.6138578; 0.8926451 -0.3799693]
b = Float32[0.0, 0.0, 0.0]
activation = relu
p = Params([Float32[0.9782329 -0.32619792; -0.11098309 -0.6138578; 0.8926451 -0.3799693], Float32[0.0, 0.0, 0.0]])


In [36]:
# we can also use CUDA to work directly with GPU. Flux provides the gpu and cpu functions as a wrapper to this that also works if there is no GPU available.
a_gpu = cu(rand(1000,1000))
b_gpu = cu(rand(1000,1000))
c_gpu = a_gpu*b_gpu
CUDA.@profile c_gpu=a_gpu*b_gpu;

In [28]:
# Some information about the GPU
device = CUDA.device()
println("number of devices = $(CUDA.ndevices())")
println("Info on first device:")
println("GPU = $(CUDA.name(device))")
println("GPU memory = $(CUDA.totalmem(device)/1024^3) Gb")
println("warpsize = $(CUDA.warpsize(device))")
println("capability = $(CUDA.capability(device))")
println("memory_pools_supported = $(CUDA.memory_pools_supported(device))")
println("uuid = $(CUDA.uuid(device))")

number of devices = 1
Info on first device:
GPU = NVIDIA GeForce RTX 3050 Ti Laptop GPU
GPU memory = 3.81298828125 Gb
warpsize = 32
capability = 8.6.0
memory_pools_supported = true
uuid = 8c480ede-6586-57c2-11ac-b9a589975f5f


In [33]:
# Layers are stacked into a Chain
NN_chain = Chain(
    Dense(2, 3, relu),
    Dense(3, 1)
)
println("Combined parameters: ", Flux.params(NN_chain))
NN_chain[1], NN_chain[2]
x1=rand(2) # 2-element input
z1=NN_chain[1](x1) # 2-element input to first layer
z2=NN_chain[2](z1) # 3-element output from first layer fed into second layer
z2_chained=NN_chain(x1) # same as above
z2, z2_chained

Combined parameters: Params([

Float32[-1.0839155 0.60505545; -0.8642427 0.89124316; -0.2969589 -0.20246455], Float32[0.0, 0.0, 0.0], Float32[0.017702471 -1.1042601 -0.64565307], Float32[0.0]])


(Float32[-0.052845508], Float32[-0.052845508])

In [40]:
# all parameters of the network as a vector
collect(Iterators.Flatten(Flux.params(NN_chain)))

13-element Vector{Float32}:
 -1.0839155
 -0.8642427
 -0.2969589
  0.60505545
  0.89124316
 -0.20246455
  0.0
  0.0
  0.0
  0.017702471
 -1.1042601
 -0.64565307
  0.0